# TP 1 - Analisis de datos comerciales para entender su Composición

## Introducción

`Dataset: Datos.csv`

Este proyecto de data science se enfoca en el análisis de una base de datos alterada de una empresa comercial que realiza ventas online. La base de datos, cuidadosamente falsificada, no guarda relación con la realidad ni toma en cuenta efemérides, lo que permite analizar las ventas en un contexto controlado y simulado, proporcionando información valiosa para realizar inferencias sin comprometer la integridad de datos reales.

### Objetivo de análisis:

El objetivo principal del proyecto es analizar la composición y distribución de las ventas contenidas en el dataset, generando gráficos y análisis numéricos para responder preguntas de interés y validar o refutar hipótesis comerciales. Este análisis permitirá generar un informe detallado para el área comercial de la empresa.

El análisis se enfoca en las siguientes hipótesis iniciales:

1. **Distribución geográfica**: Dado que la sede principal de la empresa se encuentra en Córdoba, se espera que al menos el 70% de las ventas provengan de esta localidad.
2. **Concentración de ventas en tiendas**: El 60% de las operaciones deberían estar concentradas en tres de las tiendas principales (Tiendas 1, 2 y 5).
3. **Comportamiento temporal de las ventas**: Se asume que la mayoría de las ventas ocurren durante los fines de semana.
4. **Patrones horarios**: Se estima que una cantidad considerable de ventas se realizan en horarios de siesta.

#### Gráficos y análisis de variables:

Para dar respuesta a estas hipótesis, se crearán gráficos que incluyan al menos tres variables clave, como **ubicación geográfica**, **horario de la transacción**, y **categoría del producto**. Estos gráficos permitirán realizar un diagnóstico visual de cómo se comportan las ventas en función de las preguntas de interés. Además, el análisis numérico complementará los gráficos, brindando información cuantitativa precisa sobre las tendencias observadas.

#### Identificación de valores perdidos:

Durante el proceso de análisis, se identificarán los valores perdidos en el dataset, lo que permitirá evaluar su impacto en los resultados. Se implementarán técnicas para gestionar estos valores, garantizando que no afecten de forma significativa las conclusiones del análisis.

### Diccionario de datos:

Para facilitar la comprensión y análisis del dataset, se incluye un diccionario de datos que detalla los campos más relevantes. Este diccionario abarca variables como el **origen del pedido** (marketplace o fulfillment), el **documento del cliente**, la **ciudad** y **estado** desde donde se realizó el pedido, el **tipo de dirección de entrega**, así como detalles sobre las **promociones aplicadas**, el **código de referencia del SKU**, el **precio de venta del SKU**, entre otros.

## Diccionario

* Origin:  indica si el origen del pedido es marketplace o fulfillment.
* Sequence:  es la secuencia numérica de la transacción del pedido.
* Creation Date:  es la fecha y hora en que el cliente realizó el pedido.
* Client Document:  es el número de identificación fiscal del cliente que realizó el pedido.
* UF:  es el estado, provincia, región o departamento desde el que el cliente realizó el pedido.
* City:  es la ciudad desde la que el cliente realizó el pedido.
* Address Type:  es el tipo de dirección de entrega (domicilio o recogida).
* Postal Code:  es el código postal de la dirección de entrega.
* Courier:  es la transportadora responsable de la entrega.
* Status:  es el status del pedido en el momento de exportación la plantilla.
* Coupon:  es el ID del cupón del pedido, si lo hay.
* Quantity_SKU:  es la cantidad de ítems del mismo SKU.
* Category Ids Sku:  son los ID de las categorías en los que se encuentra el SKU. Esta información se presenta siguiendo el árbol de departamento/categoría/subcategoría. Por ejemplo, si el valor es /1000003/1000016/, eso significa que el SKU está en la categoría 1000016, que a su vez forma parte del departamento 1000003.
* Reference Code:  es el código de referencia del SKU.
* SKU Selling Price:  es el precio al que se vendió el SKU realmente, ya con los descuentos o promociones aplicados.
* SKU Total Price:  es el valor total del SKU en el pedido, considerando todos os ítems del SKU.
* Total Value:  es el valor total del pedido. Atención
* Discounts Names:  son las promociones aplicadas al pedido.
* Canceled By:  es el email del usuario que realizó la cancelación del pedido.
* Corporate Document:  es el número de registro de persona jurídica, cuando el pedido es realizado por una empresa.
* SalesChannel:  es el ID de la política comercial del pedido.


## Código

### Inicializando Datos

In [19]:
# Importación de librerias
import pandas as pd
import locale

locale.setlocale(locale.LC_TIME, 'Spanish_Argentina.1252')

'Spanish_Argentina.1252'

In [28]:
# Asignacion del dataset a la variable "dataset"
dataset = pd.read_csv('./Datos/Datos.csv',sep=';')

In [29]:
# Lectura basica de la cabecera
dataset.head()

,Origin,Sequence,Creation Date,Client Document,UF,City,Address Type,Postal Code,Courrier,Status,...,Quantity_SKU,Category Ids Sku,Reference Code,SKU Selling Price,SKU Total Price,Total Value,Discounts Names,Cancelled By,Corporate Document,SalesChannel
0,Marketplace,1488581,2020-06-12 15:48:08Z,1,Buenos Aires,Olavarria,pickup,7400,Correo Sucursal,Faturado,...,1,/1/50/59/,1 - 1213,16999.0,16999.0,118098.00,x,NaN,NaN,1
1,Marketplace,1470647,2020-06-06 21:04:26Z,2,Córdoba,Córdoba,residential,5014,Correo,Faturado,...,1,/1/34/78/,2 - 0,8999.0,8999.0,63897.99,x,NaN,NaN,1
2,Marketplace,1464248,2020-06-01 22:38:00Z,3,Córdoba,Córdoba,residential,5000,Correo,Faturado,...,1,/1/34/78/,3 - 0,9629.1,9629.1,32108.20,x,NaN,NaN,1
3,Marketplace,1502447,2020-06-23 02:05:13Z,4,Buenos Aires,Victoria,pickup,1644,Correo Sucursal,Faturado,...,1,/1/34/78/,4 - 0,35799.0,35799.0,40299.00,NaN,NaN,NaN,1
4,Marketplace,1479789,2020-06-09 10:04:41Z,5,Buenos Aires,Santa Teresita,residential,7107,Correo,Faturado,...,1,/1/34/78/,4 - 0,33499.0,33499.0,67098.00,x,NaN,NaN,1


In [30]:
# Se obtiene información básica de las columnas
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24554 entries, 0 to 24553
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Origin              24554 non-null  object 
 1   Sequence            24554 non-null  int64  
 2   Creation Date       24554 non-null  object 
 3   Client Document     24554 non-null  int64  
 4   UF                  24554 non-null  object 
 5   City                24554 non-null  object 
 6   Address Type        24554 non-null  object 
 7   Postal Code         24554 non-null  int64  
 8   Courrier            24554 non-null  object 
 9   Status              24554 non-null  object 
 10  Coupon              609 non-null    object 
 11  Quantity_SKU        24554 non-null  int64  
 12  Category Ids Sku    24554 non-null  object 
 13  Reference Code      24554 non-null  object 
 14  SKU Selling Price   24554 non-null  float64
 15  SKU Total Price     24554 non-null  float64
 16  Tota

In [31]:
# Conversiones necesarias

# Convertir las columnas de identificadores a texto
dataset['Client Document'] = dataset['Client Document'].astype(str)
dataset['Postal Code'] = dataset['Postal Code'].astype(str)
dataset['Corporate Document'] = dataset['Corporate Document'].astype(str)
dataset['SalesChannel'] = dataset['SalesChannel'].astype(str)

# Convertir 'Creation Date' a datetime
dataset['Creation Date'] = pd.to_datetime(dataset['Creation Date'])

In [32]:
# Creación de las columnas necesarias para el analisis objetivo

# Extraer el día de la semana (Devuelve números: 0=Lunes, 6=Domingo)
dataset['Day of Week'] = dataset['Creation Date'].dt.dayofweek

# Extraer el nombre del día
dataset['Day Name'] = dataset['Creation Date'].dt.strftime('%A') 

# Extraer la hora
dataset['Hour'] = dataset['Creation Date'].dt.hour

In [33]:
dataset.head()

,Origin,Sequence,Creation Date,Client Document,UF,City,Address Type,Postal Code,Courrier,Status,...,SKU Selling Price,SKU Total Price,Total Value,Discounts Names,Cancelled By,Corporate Document,SalesChannel,Day of Week,Day Name,Hour
0,Marketplace,1488581,2020-06-12 15:48:08+00:00,1,Buenos Aires,Olavarria,pickup,7400,Correo Sucursal,Faturado,...,16999.0,16999.0,118098.00,x,NaN,nan,1,4,Friday,15
1,Marketplace,1470647,2020-06-06 21:04:26+00:00,2,Córdoba,Córdoba,residential,5014,Correo,Faturado,...,8999.0,8999.0,63897.99,x,NaN,nan,1,5,Saturday,21
2,Marketplace,1464248,2020-06-01 22:38:00+00:00,3,Córdoba,Córdoba,residential,5000,Correo,Faturado,...,9629.1,9629.1,32108.20,x,NaN,nan,1,0,Monday,22
3,Marketplace,1502447,2020-06-23 02:05:13+00:00,4,Buenos Aires,Victoria,pickup,1644,Correo Sucursal,Faturado,...,35799.0,35799.0,40299.00,NaN,NaN,nan,1,1,Tuesday,2
4,Marketplace,1479789,2020-06-09 10:04:41+00:00,5,Buenos Aires,Santa Teresita,residential,7107,Correo,Faturado,...,33499.0,33499.0,67098.00,x,NaN,nan,1,1,Tuesday,10


In [34]:
# Se obtiene nuevamente información básica de las columnas para verificación
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24554 entries, 0 to 24553
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Origin              24554 non-null  object             
 1   Sequence            24554 non-null  int64              
 2   Creation Date       24554 non-null  datetime64[ns, UTC]
 3   Client Document     24554 non-null  object             
 4   UF                  24554 non-null  object             
 5   City                24554 non-null  object             
 6   Address Type        24554 non-null  object             
 7   Postal Code         24554 non-null  object             
 8   Courrier            24554 non-null  object             
 9   Status              24554 non-null  object             
 10  Coupon              609 non-null    object             
 11  Quantity_SKU        24554 non-null  int64              
 12  Category Ids Sku    24554 non-nu

In [11]:
# Analisis describe
datos.describe()

,Sequence,Client Document,Postal Code,Quantity_SKU,SKU Selling Price,SKU Total Price,Total Value,Corporate Document,SalesChannel
count,2.455400e+04,24554.00000,24554.000000,24554.000000,2.455400e+04,2.455400e+04,2.455400e+04,1318.000000,24554.000000
mean,1.499497e+06,7883.27971,4354.011770,1.024232,4.455358e+04,4.492515e+04,6.718265e+04,485.680577,1.917203
std,2.056837e+04,5071.71530,2140.052713,0.245398,3.898706e+04,3.911871e+04,5.315971e+04,286.428396,1.162668
min,1.463347e+06,1.00000,1001.000000,1.000000,1.990000e+02,1.990000e+02,1.990000e+02,1.000000,1.000000
25%,1.481656e+06,3365.25000,2126.000000,1.000000,1.881000e+04,1.899925e+04,3.539900e+04,237.250000,1.000000
50%,1.500419e+06,7667.00000,5000.000000,1.000000,3.528000e+04,3.568000e+04,5.999600e+04,482.000000,1.000000
75%,1.516972e+06,12243.75000,5300.000000,1.000000,6.000000e+04,6.089900e+04,8.198475e+04,711.750000,3.000000
max,1.535422e+06,17166.00000,9600.000000,14.000000,1.119499e+06,1.119499e+06,1.119499e+06,1000.000000,5.000000


Consigna

1. Generar preguntas o hipótesis de interés sobre el dataset elegido para el proyecto final.
2. Crear gráficos que usen al menos tres variables y hacer un diagnóstico de lo que los mismos nos están diciendo.
3. Vincular los gráficos y análisis numéricos hechos hasta el momento con las preguntas de interés.
4. Identificar valores perdidos.


El análisis se enfoca en las siguientes hipótesis iniciales:

1. **Distribución geográfica**: Dado que la sede principal de la empresa se encuentra en Córdoba, se espera que al menos el 70% de las ventas provengan de esta localidad.
2. **Concentración de ventas en tiendas**: El 60% de las operaciones deberían estar concentradas en tres de las tiendas principales (Tiendas 1, 2 y 5).
3. **Comportamiento temporal de las ventas**: Se asume que la mayoría de las ventas ocurren durante los fines de semana.
4. **Patrones horarios**: Se estima que una cantidad considerable de ventas se realizan en horarios de siesta.

### Gráficos y análisis de variables:

Para dar respuesta a estas hipótesis, se crearán gráficos que incluyan al menos tres variables clave, como **ubicación geográfica**, **horario de la transacción**, y **categoría del producto**. Estos gráficos permitirán realizar un diagnóstico visual de cómo se comportan las ventas en función de las preguntas de interés. Además, el análisis numérico complementará los gráficos, brindando información cuantitativa precisa sobre las tendencias observadas.

### Identificación de valores perdidos:

Durante el proceso de análisis, se identificarán los valores perdidos en el dataset, lo que permitirá evaluar su impacto en los resultados. Se implementarán técnicas para gestionar estos valores, garantizando que no afecten de forma significativa las conclusiones del análisis.